# Performing baselie subtraction

The baseline-subtracted histograms of an acquisition with 2 active channels were obtained. The baseline value for each channel is the mean of the readings from all the events triggered ONLY by external HOLD assertion. This value is subtracted from the values of each "real" event (triggered from internal HOLD).

The dataset for this analysis is a background acquisition of 10 000 events. SIPHRA channels 2 and 14 were connected to SiPM channels 1-4 and 5-8, respectively, using charge comparator, with a threshold value of 20.

In [1]:
import ROOT
import pandas as pd
import numpy as np
import os
import sys
import ipynbname
from pathlib import Path

project_root = str(ipynbname.path().parent.parent)
sys.path.append(project_root)

from processing import SiphraAcquisition
from analysis import fit_peak_expbg

# ROOT.gROOT.SetBatch(False)
# ROOT.gEnv.SetValue("Canvas.ShowEventStatus", 1)
# ROOT.gEnv.SetValue("Canvas.ShowToolBar", 1)
# ROOT.gEnv.SetValue("Canvas.UseGL", 0)
# ROOT.gROOT.ProcessLine("gVirtualX = new TGX11();")


In [2]:
# Constants
BITS12 = 2**12
BITS9 = 2**9 # 512 typical number of bins used
# Energy emission peaks in MeV
K40_MEV = 1.460
TL208_MEV = 2.614
CS137_MEV = 0.661
colors = [ROOT.kRed, ROOT.kBlue, ROOT.kGreen, ROOT.kOrange, ROOT.kViolet, ROOT.kYellow, ROOT.kSpring, ROOT.kCyan,]

In [3]:
# Datasets
acq_notSubtracted = SiphraAcquisition(project_root+'/data/260203/1_SiPM_ChannelsTest_Ch1-4_Ch2_Ch5-8_Ch14_QT_Thr20_Hys0_Background.csv',
                                   active_chs=[2,14],
                                   exposure_sec=1)
acq_subtracted = SiphraAcquisition(project_root+'/data/260203/1_datdecodertest_subtractBaseline.csv',
                                      active_chs=[2,14],
                                      exposure_sec=1)

In [4]:
if ROOT.gROOT.FindObject('cv'):
    ROOT.gROOT.FindObject('cv').Close()

SUMMED_XR=5000 # Right limit of the summed channel's x-axis

c = ROOT.TCanvas('cv', 'cv', 1200, 800)
ROOT.gStyle.SetOptStat(11)
ROOT.gStyle.SetStatFontSize(0.03)
ROOT.gStyle.SetStatW(0.16)

names=['BL subt.', 'No correction']
hists = []

lg = ROOT.TLegend(0.48, 0.61, 0.75, 0.83)

Yinit = 0.82 # For stat boxes

for idx, (acq, name) in enumerate(zip([acq_subtracted, acq_notSubtracted], names)):
    # print(f"Current file: {acq.filepath.name}")
    ch = 'Summed' #acq.active_chs[0]
    hist = ROOT.TH1F(f"{ch} {name}", "", int(BITS9*SUMMED_XR/BITS12), 0, SUMMED_XR)
    hist_singlech = ROOT.TH1F(f"Ch. {acq.active_chs[0]} {name}", "", BITS9, 0, BITS12)
    hist.Fill(acq[ch]/len(acq.active_chs))
    hist_singlech.Fill(acq[acq.active_chs[0]])
    hist.Scale(1/acq.exposure)
    hist_singlech.Scale(1/acq.exposure)

    hist_singlech2 = ROOT.TH1F(f"Ch. {acq.active_chs[1]} {name}", "", BITS9, 0, BITS12)
    hist_singlech2.Fill(acq[acq.active_chs[1]])
    hist_singlech2.Scale(1/acq.exposure)
    #Preeting up..
    hist.GetXaxis().SetTitle("ADC channel number")
    hist.GetYaxis().SetTitle(r"Counts")
    hist.SetLineColor(colors[idx])
    hist.SetTitle("Baseline subtraction comparison")
    # hist.GetYaxis().SetRangeUser(0, 1e4)
    lg.SetHeader("SIPHRA Channel")
    lg.AddEntry(hist, f"{ch} {name}", 'l')
    hists.append(hist)
    hists[-1].Draw('sames hist')
    # hist_singlech.GetXaxis().SetTitle("ADC channel number")
    # hist_singlech.GetYaxis().SetTitle(r"Count rate (s^{-1})")
    hist_singlech.SetLineColor(colors[idx + 2]+2)
    hist_singlech2.SetLineColor(colors[idx + 4]+1)
    # hist_singlech.SetTitle("Baseline subtraction comparison")
    lg.AddEntry(hist_singlech, f"Ch. {acq.active_chs[0]} {name}", 'l')
    lg.AddEntry(hist_singlech2, f"Ch. {acq.active_chs[1]} {name}", 'l')
    hists.append(hist_singlech)
    hists[-1].Draw('sames hist')
    hists.append(hist_singlech2)
    hists[-1].Draw('sames hist')
c.SetLogy()
ROOT.gPad.Update()
for i, sp in enumerate(hists):
    st = sp.FindObject("stats")
    # print(type(st))
    st.SetY1NDC(Yinit - i*0.08)
    st.SetY2NDC(0.1)
lg.Draw()
c.Draw()
ROOT.gPad.Modified()
ROOT.gPad.Update()

In [13]:
if ROOT.gROOT.FindObject('cv_4plots'):
    ROOT.gROOT.FindObject('cv_4plots').Close()

canvas4 = ROOT.TCanvas('cv_4plots', 'cv_4plots', 1600, 1200)
canvas4.Divide(2,2)

legends = [ROOT.TLegend(0.48, 0.61, 0.75, 0.83) for _ in range(4)]

canvas4.cd(1)
hists[0].Draw('hist')
hists[3].Draw('sames hist')
legends[0].AddEntry(hists[0], "Baseline subtracted", 'l')
legends[0].AddEntry(hists[3], "Not corrected", 'l')
legends[0].SetHeader("\'Summed\' channel spectra")
legends[0].Draw()
canvas4.cd(1).SetLogy()

canvas4.cd(2)
hists[1].SetXTitle("ADC channel number")
hists[1].SetYTitle("Counts")
hists[1].Draw('hist')
hists[4].Draw('sames hist')
legends[1].AddEntry(hists[1], "Baseline subtracted", 'l')
legends[1].AddEntry(hists[4], "Not corrected", 'l')
legends[1].SetHeader("Ch. 2 spectra")
legends[1].Draw()
canvas4.cd(2).SetLogy()

canvas4.cd(3)
hists[2].SetXTitle("ADC channel number")
hists[2].SetYTitle("Counts")
hists[2].Draw('hist')
hists[5].Draw('sames hist')
legends[2].AddEntry(hists[2], "Baseline subtracted", 'l')
legends[2].AddEntry(hists[5], "Not corrected", 'l')
legends[2].SetHeader("Ch. 14 spectra")
legends[2].Draw()
canvas4.cd(3).SetLogy()

canvas4.cd(4)
h0_zoom = hists[0].Clone("hists_0_zoomed")
h0_zoom.GetXaxis().SetRangeUser(0,BITS12)
h0_zoom.Draw('hist')
hists[1].Draw('sames hist')
hists[2].Draw('sames hist')
legends[3].AddEntry(h0_zoom, "\'Summed\'", 'l')
legends[3].AddEntry(hists[1], "Ch. 2", 'l')
legends[3].AddEntry(hists[2], "Ch. 14", 'l')
legends[3].SetHeader("Corrected channels")
legends[3].Draw()
canvas4.cd(4).SetLogy()

canvas4.Draw()

In [6]:
print("ADC channels range:\n"
      f"{"":->50}\n"
      "Channel\t\tBL-subtracted\tNo correction\n"
      f"{"":->50}\n"
      f"Summed\t\t{acq_subtracted['s'].min()/2} - {acq_subtracted['s'].max()/2}\t{acq_notSubtracted['s'].min()/2} - {acq_notSubtracted['s'].max()/2}\n"
      f"Ch. 2\t\t{acq_subtracted[2].min()} - {acq_subtracted[2].max()}\t{acq_notSubtracted[2].min()} - {acq_notSubtracted[2].max()}\n"
      f"Ch. 14\t\t{acq_subtracted[14].min()} - {acq_subtracted[14].max()}\t{acq_notSubtracted[14].min()} - {acq_notSubtracted[14].max()}\n")

print("\n\n"
      "Number of values above single channel range in \'Summed\' channel:\n"
      f"{"":->30}\n"
      "BL-subtracted\tNo correction\n"
      f"{"":->30}\n"
      f"{len(acq_subtracted['s'][acq_subtracted['s'][:]/2>BITS12]):^13}\t{len(acq_notSubtracted['s'][acq_notSubtracted['s'][:]/2>BITS12]):^13}")

ADC channels range:
--------------------------------------------------
Channel		BL-subtracted	No correction
--------------------------------------------------
Summed		0.0 - 3965.0	935.5 - 4880.5
Ch. 2		0.0 - 3951.0	51.0 - 4095.0
Ch. 14		0.0 - 3979.0	0.0 - 4095.0



Number of values above single channel range in 'Summed' channel:
------------------------------
BL-subtracted	No correction
------------------------------
      0      	     22      


The baseline subtraction is performed directly at the conversion script. No averaging over the number of channels is carried out by the conversion script, but the above spectra and data correspond to values averaged over the number of active channels (2).

The values of the baselines of all the channels are between 98 and 150.

The range of the summed channel does not exceed the maximum single-channel value (4096) after baseline correction.

**<span style="color:red">Questions:</span>**
- Is this baseline correction enough for pedestal subtraction?
- What about the large peak at the lower range of the spectrum? Is it still originating from noise at this stage?

# Calibration of corrected spectrum with the backgrond $^{40}$K and $^{208}$Tl peaks

The current process of calibration to actual energies is shown here. The two peaks of $^{40}$K and $^{208}$Tl present in the background are used for this purpose. The emission lines of these nuclides are located at 1.460 MeV and 2.614 MeV, respectively. First, both peaks are fitted to a Gaussian with exponentially-decaying background in the baseline-subtracted background histogram. The Gaussian means are taken as the actual locations of the emission lines. The raw data is linearly transformed to aligh with these two points.

To verify the calibration, we use the same calibration parameters to fit a spectrum taken in the pressence of a sample of $^{137}$Cs. After background subtraction, it is expected that the mean of the Gaussian fit of the resulting peak, lands at 0.661MeV.

In [76]:
# Datasets
folder = Path(project_root)/'data/260209'
times_lst = [0, 119.634, 161.062, 148.920, 164.828, 34.836, 80.792, 54.386, 90.837, 24.645, 24.144, 24.606, 23.381, 24.720, 24.916, 24.848, 24.934] # Index coincide with starting number of file name

# Select SiPM channels 5-8, SIPHRA channel 14 -> gives cleaner signal
# Background is file with #4, source is file with #16, with preffix 'BLsubtr_'
BGIDX = 4; SRCIDX=16; ACTIVECH=14
bgfile = sorted(folder.glob(f'BLsubtr_{BGIDX}*.csv'))[0]
srcfile = sorted(folder.glob(f'BLsubtr_{SRCIDX}*.csv'))[0]

acq_bg = SiphraAcquisition(bgfile, active_chs=ACTIVECH, exposure_sec=times_lst[BGIDX])
acq_src = SiphraAcquisition(srcfile, active_chs=ACTIVECH, exposure_sec=times_lst[SRCIDX])
colors_clb = [colors[1]-1, colors[3]-1, colors[0]-2] # colors for BG, signal and clean, respectively
NORMFACTOR = acq_src.exposure/(acq_bg.exposure)

In [8]:
print("ADC channels range after baseline subtraction:\n"
      "CHANNEL 14\n"
      f"{"":->30}\n"
      "Data set\tRange\n"
      f"{"":->30}\n"
      f"Background\t{acq_bg[ACTIVECH].min()} - {acq_bg[ACTIVECH].max()}\n"
      f"Source\t\t{acq_src[ACTIVECH].min()} - {acq_src[ACTIVECH].max()}\n\n"
      "SUMMED CHANNEL\n"
      f"{"":->30}\n"
      "Data set\tRange\n"
      f"{"":->30}\n"
      f"Background\t{acq_bg['s'].min()} - {acq_bg['s'].max()}\n"
      f"Source\t\t{acq_src['s'].min()} - {acq_src['s'].max()}\n")

ADC channels range after baseline subtraction:
CHANNEL 14
------------------------------
Data set	Range
------------------------------
Background	0.0 - 3924.0
Source		0.0 - 3797.0

SUMMED CHANNEL
------------------------------
Data set	Range
------------------------------
Background	0.0 - 3924.0
Source		0.0 - 3797.0



Baseline subtraction is effectively filtering out noise from inactive channels. It is noteworthy that, when subtracting single channels, the upper ADC channels will be empty, hence there is a loss of dynamic range due to baseline subtraction.

In [43]:
# Raw histograms
hist_bg = ROOT.TH1F("Background", "", BITS9, 0, BITS12)
hist_bg.Fill(acq_bg[ACTIVECH])
hist_bg.Scale(NORMFACTOR) # Normalize counts to the source exposure time

hist_src = ROOT.TH1F("Signal 137Cs", "", BITS9, 0, BITS12)
hist_src.Fill(acq_src[ACTIVECH])

# Bacground subtraction
hist_clean = hist_src.Clone("137Cs no BG")
hist_clean.Add(hist_bg, -1)
# Removing negative bins
for i in range(1, hist_clean.GetNbinsX() + 1): # bin 0 is the underflow
    if hist_clean.GetBinContent(i) < 0: hist_clean.SetBinContent(i, 0)

Warning in <TROOT::Append>: Replacing existing TH1: Background (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Signal 137Cs (Potential memory leak).


In [62]:
# Plot all the histograms
if ROOT.gROOT.FindObject('cv1'):
    ROOT.gROOT.FindObject('cv1').Close()
cv1 = ROOT.TCanvas("cv1", "cv1", 1600, 800)
cv1.Divide(2,1)

lg1 = ROOT.TLegend(0.48, 0.71, 0.75, 0.83)
cv1.cd(1)
hist_bg.GetXaxis().SetTitle("ADC Channel")
hist_bg.GetYaxis().SetTitle("Normalized counts")
hist_bg.SetLineColor(colors_clb[0])
hist_bg.SetFillColor(colors_clb[0])
hist_src.SetLineColor(colors_clb[1])
hist_src.SetFillColorAlpha(colors_clb[1], 0.8)
lg1.AddEntry(hist_bg, "Background", "l")
lg1.AddEntry(hist_src, r"Signal ^{137}Cs", "l")
hist_bg.Draw("hist")
hist_src.Draw("sames hist")
lg1.Draw()
cv1.cd(1).SetLogy()
hist_bg.GetYaxis().SetRangeUser(0,1e4)


# lg2 = ROOT.TLegend(0.48, 0.71, 0.75, 0.83)
cv1.cd(2)
hist_clean.GetXaxis().SetTitle("ADC Channel")
hist_clean.GetYaxis().SetTitle("Normalized counts")
hist_clean.SetLineColor(colors_clb[2])
hist_clean.SetFillColor(colors_clb[2])
hist_clean.Draw("hist")
cv1.cd(2).SetLogy()

cv1.Draw()

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [72]:
# Fit background peaks
xrange_k40 = (600,1000)
nrm_k40_i = 55
mean_k40_i = 740
xrange_tl208 = (1000,1400)
nrm_tl208_i = 7
mean_tl208_i = 1210
hist_bg_fit = hist_bg.Clone()
hist_bg_fit.GetYaxis().SetRangeUser(0,1e3)
hist_bg.SetTitle(r"^{40}K and ^{208}Tl peaks fitting")
fit_fn_k40, fit_res_k40 = fit_peak_expbg(hist_bg_fit, name="K40_peak", xl=xrange_k40[0], xr=xrange_k40[1], norm=nrm_k40_i, mean=mean_k40_i, showFit=True)
fit_fn_tl208, fit_res_tl208 = fit_peak_expbg(hist_bg_fit, name="Tl208_peak", xl=xrange_tl208[0], xr=xrange_tl208[1], norm=nrm_tl208_i, mean=mean_tl208_i, showFit=True)

cv_fit = ROOT.TCanvas("cv_fit", "cv_fit", 800,600)
hist_bg_fit.SetFillStyle(0)
hist_bg_fit.Draw("hist")
cv_fit.SetLogy()
cv_fit.Draw()

****************************************
Minimizer is Minuit2 / Migrad
MinFCN                    =      3.07992
Chi2                      =      6.15985
NDf                       =           45
Edm                       =  4.33544e-06
NCalls                    =          378
Const                     =      967.676   +/-   315.198     
Denom                     =      204.714   +/-   16.8531     
Norm                      =      26.0236   +/-   2.92104     
Mean                      =      748.998   +/-   10.0065     
Sigma                     =      69.1169   +/-   10.4622     
****************************************
Minimizer is Minuit2 / Migrad
MinFCN                    =      4.22284
Chi2                      =      8.44567
NDf                       =           45
Edm                       =  1.92878e-06
NCalls                    =          607
Const                     =      65.7641   +/-   57.5374     
Denom                     =      441.486   +/-   153.51      
Norm          

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cv_fit
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [103]:
# Calibrate data and get calibrated histograms
K40FIT = fit_fn_k40.GetParameter('Mean')
TL208FIT = fit_fn_tl208.GetParameter('Mean')
clb_slope = (TL208_MEV - K40_MEV)/(TL208FIT - K40FIT)
clb_const = K40_MEV - clb_slope * K40FIT

data_bg_clb = acq_bg[ACTIVECH]*clb_slope + clb_const
data_src_clb = acq_src[ACTIVECH]*clb_slope + clb_const

CLBHISTS_XR = np.max([data_bg_clb.max(), data_src_clb.max()])

hist_bg_clb = ROOT.TH1F("Calibrated background", "Calibrated Histograms", BITS9, 0, CLBHISTS_XR)
hist_src_clb = ROOT.TH1F("Calibrated signal", "Calibrated Histograms", BITS9, 0, CLBHISTS_XR)

hist_bg_clb.Fill(data_bg_clb)
hist_bg_clb.Scale(NORMFACTOR)

hist_src_clb.Fill(data_src_clb)

hist_clean_clb = hist_src_clb.Clone("Calibrated signal no BG")
hist_clean_clb.Add(hist_bg_clb, -1)

for hist, clr in zip([hist_bg_clb, hist_src_clb, hist_clean_clb], colors_clb):
    hist.GetXaxis().SetTitle("Energy (MeV)")
    hist.GetYaxis().SetTitle("Normalized counts")
    hist.SetLineColor(clr)

Warning in <TROOT::Append>: Replacing existing TH1: Calibrated background (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Calibrated signal (Potential memory leak).


In [101]:
# Plot all the histograms
if ROOT.gROOT.FindObject('cv1'):
    ROOT.gROOT.FindObject('cv1').Close()
cv1 = ROOT.TCanvas("cv1", "cv1", 1600, 800)
cv1.Divide(2,1)

lg1 = ROOT.TLegend(0.48, 0.71, 0.75, 0.83)
cv1.cd(1)
hist_bg_clb.SetFillColor(colors_clb[0])
hist_src_clb.SetFillColorAlpha(colors_clb[1], 0.8)
lg1.AddEntry(hist_bg_clb, "Background", "l")
lg1.AddEntry(hist_src_clb, r"Signal ^{137}Cs", "l")
hist_bg_clb.Draw("hist")
hist_src_clb.Draw("sames hist")
lg1.Draw()
cv1.cd(1).SetLogy()
hist_bg_clb.GetYaxis().SetRangeUser(0,1e4)


# lg2 = ROOT.TLegend(0.48, 0.71, 0.75, 0.83)
cv1.cd(2)
hist_clean_clb.SetFillColor(colors_clb[2])
hist_clean_clb.Draw("hist")
cv1.cd(2).SetLogy()

cv1.Draw()

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [115]:
res = hist_clean_clb.Fit("gaus", "L S", "", 0, 2)

****************************************
Minimizer is Minuit2 / Migrad
MinFCN                    =      5628.78
Chi2                      =      11257.6
NDf                       =          112
Edm                       =  1.88824e-07
NCalls                    =          124
Constant                  =      4002.29   +/-   17.4275     
Mean                      =     0.479418   +/-   0.000489701 
Sigma                     =     0.136089   +/-   0.000348374  	 (limited)


In [129]:
cv_fit = ROOT.TCanvas("cv_fit", "cv_fit", 800,600)
# hist_bg_fit.SetFillStyle(0)
hist_clean_clb.Draw("hist")
hist_clean_clb.SetFillColor(colors_clb[2]-6)
cv_fit.SetLogy()
cv_fit.Draw()


err = (CS137_MEV - hist_clean_clb.GetFunction("gaus").GetParameter("Mean"))/CS137_MEV
print(f"Error in the location of Cs peak: {err*100:.1f}%")

Error in the location of Cs peak: 27.5%


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cv_fit


In [7]:
import numpy as np
sigma = 0.136089
FWHM = np.sqrt(8*np.log(2))*sigma
print(f"{FWHM=:.2f}   Energy resolution at 480MeV: {FWHM*100/0.479418:.2f}%")

FWHM=0.32   Energy resolution at 480MeV: 66.84%
